In [ ]:
pip install codecarbon

In [ ]:
from transformers import AutoTokenizer, AutoModel
from calflops import calculate_flops
import json
from tqdm.auto import tqdm
import time
import torch
from datasets import load_dataset
from codecarbon import EmissionsTracker

In [ ]:
token = ""

In [ ]:
def get_time(model, tokenizer, dataset):
    start = time.time()
    with torch.no_grad():
        for sample in tqdm(dataset):
            inputs = tokenizer(sample, return_tensors="pt", truncation=True, max_length=512).to('cuda')
            outputs = model(**inputs)
    end = time.time()
    return end-start

In [ ]:
def get_speed_ratio(baseline_time, model, tokenizer, dataset):
    time = get_time(model, tokenizer, dataset)
    return baseline_time/time

In [ ]:
def calculaFlopsEParams(modelo, tokenizer):
    flops, macs, params = calculate_flops(model=modelo, input_shape=(1, 512), transformer_tokenizer=tokenizer)
    return flops, macs, params  

In [ ]:
def test_time_inference(model, tokenizer, dataset):
    start = time.time()
    with torch.no_grad():
        for sample in tqdm(dataset):
            inputs = tokenizer(sample, return_tensors="pt", truncation=True, max_length=512).to('cuda')
            outputs = model(**inputs)
    end = time.time()
    
    total_samples = len(dataset)
    total_time = end - start
    samples_per_second = total_samples / total_time
    print(f"Samples per second: {samples_per_second}")
    return samples_per_second

In [ ]:
resultado_metricas = []
dataset = load_dataset("tcepi/bidCorpus_raw", "bidCorpus_gold")
modelos = ["tcepi/HelBERT-base-uncased-fs", "tcepi/distilHelBERT", "tcepi/HelBERT-uncased-fs-lsg"]

for model_name in modelos:
    tokenizer = AutoTokenizer.from_pretrained(model_name, use_auth_token=token)
    modelo = AutoModel.from_pretrained(model_name, use_auth_token=token)
    modelo.resize_token_embeddings(len(tokenizer))
    tracker = EmissionsTracker()
    tracker.start()
    time_inference = test_time_inference(modelo.to('cuda'), tokenizer, dataset['train']['text'])
    emissions: float =  tracker.stop()
    model_size = sum(param.numel() for param in modelo.parameters()) * modelo.dtype.itemsize / (1024**2)
    flops, macs, params = calculaFlopsEParams(modelo.to('cpu'), tokenizer)
    resultado_metricas.append({"modelo": model_name, "flops": flops, "carbon_footprint": "{} kg".format(emissions), "number_of_params": params,
                               "model_size": model_size, "samples_per_second": "{:.2f}/s".format(time_inference)})
    

In [ ]:
# Calculando o time ratio
for indice_result in range(len(resultado_metricas)):
    if indice_result == 0:
        resultado_metricas[indice_result]['speed_ratio'] = "1.00x"
    else:
        resultado_metricas[indice_result]['speed_ratio'] = "{:.2f}x".format(float(resultado_metricas[indice_result]['samples_per_second'].split("/")[0]) / float(resultado_metricas[0]['samples_per_second'].split("/")[0]))

In [ ]:
json.dump(resultado_metricas, open("metricas_modelos.json", "w"), indent=4, ensure_ascii=True)